In [4]:
import os
import re
import sklearn
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.metrics import *
from sklearn.linear_model import *
from sklearn.model_selection import *

pd.set_option('display.max_columns', None)

In [5]:
spread_df=[]
years_list = [19,18,17,16,15,14,13,12,11,10,'09','08','07','06','05','04','03']
year_int = 2020
for year in years_list:
    #Note: 18 refers to 2019 and so on
    temp_spread_df = pd.read_csv("C:/Users/FLUXNATURE/Desktop/New Kaggle world/Point Spread/ncaabb" + str(year) + ".csv")
    temp_spread_df['Season'] = year_int
    year_last = year
    year_int = year_int-1    
    if year==19:
        spread_df = temp_spread_df
    else:
        spread_df = spread_df.append(temp_spread_df)

In [6]:
spread_df['home'] = spread_df['home'].str.lower()
spread_df['road'] = spread_df['road'].str.lower()

In [7]:
spread_df.head(3)

,date,home,hscore,road,rscore,line,lineavg,linesag,linesage,linesagp,linesaggm,linemoore,lineopen,linedok,linefox,std,linepugh,linedonc,neutral,linetalis,lineespn,linepir,linepiw,linepib,line7ot,lineer,linedd,linemassey,linedunk,lineround,lineteamrnks,Season,lineashby,lineprophet,lineseven,linepig,linepom,linemass,linerpi,linegreen,lineash,lineargh,linemarkov,linesauce
0,11/05/2019,alabama,NaN,pennsylvania,7.5,7.5,10.56,12.40,12.40,12.40,12.40,6.72,7.5,9.74,9.0,1.92,9.60,10.8,0.0,8.2,NaN,NaN,NaN,NaN,2.0,NaN,NaN,9.62,11.36,NaN,12.7,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11/05/2019,arkansas,NaN,rice,18,18,19.23,18.58,18.58,18.58,18.58,16.98,18,19.80,23.0,1.65,20.25,18.5,0.0,21.3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,18.76,20.03,NaN,17.1,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11/05/2019,auburn,NaN,georgia southern,14,14,15.42,15.72,15.72,15.72,15.72,5.97,14.5,15.67,20.0,3.57,17.79,15.4,0.0,16.4,NaN,NaN,NaN,NaN,2.0,NaN,NaN,18.83,NaN,NaN,12.1,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Match Team IDs to data from NCAAM

In [8]:
teams_df = pd.read_csv('C:/Users/FLUXNATURE/Desktop/New Kaggle world/NCAAM/MTeamSpellings.csv', sep='\,', engine='python')

In [9]:
teams_df.head(3)

,TeamNameSpelling,TeamID
0,a&m-corpus chris,1394
1,a&m-corpus christi,1394
2,abilene chr,1101


In [10]:
team_list = ['home','road']
for team in team_list:
    spread_df = pd.merge(spread_df, teams_df, left_on=[team], right_on = ['TeamNameSpelling'], how='left')
    if team=='home':
        spread_df.rename(columns={'TeamID': 'HTeamID'}, inplace=True)
    if team=='road':
        spread_df.rename(columns={'TeamID': 'RTeamID'}, inplace=True)
    spread_df = spread_df.drop(['TeamNameSpelling'], axis=1)

muldifying home and road to team and score

see implemetation below 

In [11]:
spread_df.loc[spread_df['hscore']>spread_df['rscore'], 'WTeamID'] = spread_df['HTeamID']
spread_df.loc[spread_df['hscore']<spread_df['rscore'], 'LTeamID'] = spread_df['HTeamID']

spread_df.loc[spread_df['hscore']>spread_df['rscore'], 'WScore'] = spread_df['hscore']
spread_df.loc[spread_df['hscore']<spread_df['rscore'], 'LScore'] = spread_df['hscore']

In [12]:
spread_df.loc[spread_df['rscore']>spread_df['hscore'], 'WTeamID'] = spread_df['RTeamID']
spread_df.loc[spread_df['rscore']<spread_df['hscore'], 'LTeamID'] = spread_df['RTeamID']

spread_df.loc[spread_df['rscore']>spread_df['hscore'], 'WScore'] = spread_df['rscore']
spread_df.loc[spread_df['rscore']<spread_df['hscore'], 'LScore'] = spread_df['rscore']

In [13]:
spread_df.replace([np.inf, -np.inf], np.nan).dropna(subset=['WTeamID','LTeamID','WScore','LScore','line'])
spread_df = spread_df[spread_df['WScore'] != "."]
spread_df = spread_df[spread_df['LScore'] != "."]
spread_df = spread_df[spread_df['line'] != "."]

In [14]:
spread_df['line'].value_counts(dropna=False)

3.0      1730
3.5      1720
2.5      1705
5.5      1699
5.0      1697
         ... 
-79.5       1
-20.5       1
-60.0       1
-77.0       1
29.5        1
Name: line, Length: 271, dtype: int64

In [15]:
#drop spreads with bad coverage
spread_df = spread_df.drop(['line7ot','lineargh','lineash','lineashby','linedd','linedunk','lineer','linegreen','linemarkov','linemass','linepib','linepig','linepir','linepiw','linepom','lineprophet','linerpi','lineround','linesauce','lineseven','neutral','lineteamrnks','linetalis','lineespn','linemassey','linedonc','linesaggm','std','linepugh','linefox','linedok','lineopen'], axis=1)

In [16]:
spread_df.tail()

,date,home,hscore,road,rscore,line,lineavg,linesag,linesage,linesagp,linemoore,Season,HTeamID,RTeamID,WTeamID,LTeamID,WScore,LScore
64236,03/30/2004,iowa st.,81,rutgers,84,2.0,1.22,1.50,1.25,1.74,1.61,2004,1235.0,1353.0,1353.0,1235.0,84,81
64237,04/01/2004,michigan,62,rutgers,55,5.5,2.06,1.51,0.46,2.44,4.47,2004,1276.0,1353.0,1276.0,1353.0,62,55
64238,04/03/2004,duke,78,connecticut,79,-2.0,2.85,2.85,2.24,3.94,1.43,2004,1181.0,1163.0,1163.0,1181.0,79,78
64239,04/03/2004,oklahoma st.,67,georgia tech,67,4.5,2.05,1.99,3.58,0.52,3.08,2004,1329.0,1210.0,NaN,NaN,NaN,NaN
64240,04/05/2004,connecticut,82,georgia tech,73,5.0,1.65,1.12,1.31,0.91,4.57,2004,1163.0,1210.0,1163.0,1210.0,82,73


In [17]:
spread_df.dropna(subset=['line'])

,date,home,hscore,road,rscore,line,lineavg,linesag,linesage,linesagp,linemoore,Season,HTeamID,RTeamID,WTeamID,LTeamID,WScore,LScore
0,11/05/2019,alabama,NaN,pennsylvania,7.5,7.5,10.56,12.40,12.40,12.40,6.72,2020,1104.0,1335.0,NaN,NaN,NaN,NaN
1,11/05/2019,arkansas,NaN,rice,18,18,19.23,18.58,18.58,18.58,16.98,2020,1116.0,1349.0,NaN,NaN,NaN,NaN
2,11/05/2019,auburn,NaN,georgia southern,14,14,15.42,15.72,15.72,15.72,5.97,2020,1120.0,1204.0,NaN,NaN,NaN,NaN
3,11/05/2019,byu,NaN,cal st. fullerton,15.5,15.5,13.72,15.69,15.69,15.69,8.86,2020,1140.0,1168.0,NaN,NaN,NaN,NaN
4,11/05/2019,baylor,NaN,central arkansas,26.5,26.5,24.66,25.88,25.88,25.88,16.69,2020,1124.0,1146.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64236,03/30/2004,iowa st.,81,rutgers,84,2.0,1.22,1.50,1.25,1.74,1.61,2004,1235.0,1353.0,1353.0,1235.0,84,81
64237,04/01/2004,michigan,62,rutgers,55,5.5,2.06,1.51,0.46,2.44,4.47,2004,1276.0,1353.0,1276.0,1353.0,62,55
64238,04/03/2004,duke,78,connecticut,79,-2.0,2.85,2.85,2.24,3.94,1.43,2004,1181.0,1163.0,1163.0,1181.0,79,78
64239,04/03/2004,oklahoma st.,67,georgia tech,67,4.5,2.05,1.99,3.58,0.52,3.08,2004,1329.0,1210.0,NaN,NaN,NaN,NaN


Write the data to a csv

In [18]:
spread_df.to_csv('spreads_all.csv', index=False)